In [1]:
import pandas as pd
import numpy as np
from googletrans import Translator  

In [2]:
df = pd.read_csv("competition_dataset/dev.tsv", sep="\t")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120744 entries, 0 to 120743
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   country      120739 non-null  object 
 1   description  120744 non-null  object 
 2   designation  84226 non-null   object 
 3   province     120739 non-null  object 
 4   region_1     100736 non-null  object 
 5   region_2     48736 non-null   object 
 6   variety      120744 non-null  object 
 7   winery       120744 non-null  object 
 8   quality      120744 non-null  float64
dtypes: float64(1), object(8)
memory usage: 8.3+ MB


In [4]:
df

,country,description,designation,province,region_1,region_2,variety,winery,quality
0,France,"A creamed pear wine, with an attractive tang o...",Brut Blanc de Blancs,Alsace,Crémant d'Alsace,NaN,Pinot Blanc,Lucien Albrecht,45.0
1,US,"Simple and dry, this Cabernet has modest black...",NaN,California,Paso Robles,Central Coast,Cabernet Sauvignon,Castle Rock,31.0
2,US,"This lovely wine captures the floral, perfumed...",NaN,Oregon,Willamette Valley,Willamette Valley,Gewürztraminer,Château Bianca,35.0
3,Portugal,"The aromas are the thing here, as so often wit...",NaN,Alentejano,NaN,NaN,Touriga Nacional,Herdade do Esporão,41.0
4,Italy,"This is an interesting, outright strange wine ...",Natì,Southern Italy,Pompeiano,NaN,Coda di Volpe,Sorrentino,37.0
...,...,...,...,...,...,...,...,...,...
120739,Italy,This is a lovely Sauvignon with very character...,NaN,Northeastern Italy,Friuli,NaN,Sauvignon,Plozner,40.0
120740,Spain,"Opens somewhat clipped and choppy, with citrus...",Reserva,Northern Spain,Rioja,NaN,Tempranillo Blend,Ramón Bilbao,41.0
120741,Australia,Surprisingly complete and harmonious for a Rie...,NaN,South Australia,Adelaide Hills,NaN,Riesling,Wagtail,26.0
120742,Germany,Broad aromas of ripe yellow fruit and a hint o...,Spätlese,Mosel,NaN,NaN,Riesling,Pamona Cellars,26.0


In [5]:
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
print(df["designation"].value_counts())

Reserve                         2189
Reserva                         1436
Estate                          1245
Barrel sample                   1069
Riserva                          609
                                ... 
Pur Chardonnay                     1
Sonoma County                      1
Loner W11-B                        1
Cemetery Vineyard Indigenous       1
Mezzan                             1
Name: designation, Length: 27800, dtype: int64


In [6]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df["designation"].value_counts()[:100])

Reserve                      2189
Reserva                      1436
Estate                       1245
Barrel sample                1069
Riserva                       609
Barrel Sample                 505
Brut                          483
Crianza                       398
Estate Grown                  362
Estate Bottled                318
Dry                           303
Old Vine                      277
Gran Reserva                  263
Brut Rosé                     208
Extra Dry                     194
Vieilles Vignes               179
Bien Nacido Vineyard          152
Rosé                          141
Late Bottled Vintage          136
Réserve                       136
Unoaked                       134
Single Vineyard               126
Late Harvest                  125
Tradition                     117
Vintage                       116
Barrel Select                 113
Grand Reserve                 107
Classic                       101
Estate Reserve                 98
Tinto         

In [7]:
#df[df["designation_tr"] == "reserva" | df["designation_tr"] == "riserva" | df["designation_tr"] == "rèserve"]["designation_tr"] = "reserve"
df["designation_tr"] = df["designation"].apply(lambda d: d.lower() if pd.isna(d) == False else d)
df["designation_tr"] = df["designation_tr"].replace(["reserva","riserva","rèserve"],["reserve","reserve","reserve"])


In [8]:
df

,country,description,designation,province,region_1,region_2,variety,winery,quality,designation_tr
0,France,"A creamed pear wine, with an attractive tang o...",Brut Blanc de Blancs,Alsace,Crémant d'Alsace,NaN,Pinot Blanc,Lucien Albrecht,45.0,brut blanc de blancs
1,US,"Simple and dry, this Cabernet has modest black...",NaN,California,Paso Robles,Central Coast,Cabernet Sauvignon,Castle Rock,31.0,NaN
2,US,"This lovely wine captures the floral, perfumed...",NaN,Oregon,Willamette Valley,Willamette Valley,Gewürztraminer,Château Bianca,35.0,NaN
3,Portugal,"The aromas are the thing here, as so often wit...",NaN,Alentejano,NaN,NaN,Touriga Nacional,Herdade do Esporão,41.0,NaN
4,Italy,"This is an interesting, outright strange wine ...",Natì,Southern Italy,Pompeiano,NaN,Coda di Volpe,Sorrentino,37.0,natì
...,...,...,...,...,...,...,...,...,...,...
120739,Italy,This is a lovely Sauvignon with very character...,NaN,Northeastern Italy,Friuli,NaN,Sauvignon,Plozner,40.0,NaN
120740,Spain,"Opens somewhat clipped and choppy, with citrus...",Reserva,Northern Spain,Rioja,NaN,Tempranillo Blend,Ramón Bilbao,41.0,reserve
120741,Australia,Surprisingly complete and harmonious for a Rie...,NaN,South Australia,Adelaide Hills,NaN,Riesling,Wagtail,26.0,NaN
120742,Germany,Broad aromas of ripe yellow fruit and a hint o...,Spätlese,Mosel,NaN,NaN,Riesling,Pamona Cellars,26.0,spätlese


In [16]:
# rimuovo regioni doppie e creo regione unica
df.loc[df["region_1"] == df["region_2"], "region_2"] = ""
df["region_complete"] = df[["region_1","region_2"]].fillna(" ").agg(' '.join, axis=1)

,country,description,designation,province,region_1,region_2,variety,winery,quality,designation_tr,region_complete
0,France,"A creamed pear wine, with an attractive tang o...",Brut Blanc de Blancs,Alsace,Crémant d'Alsace,NaN,Pinot Blanc,Lucien Albrecht,45.0,brut blanc de blancs,Crémant d'Alsace
1,US,"Simple and dry, this Cabernet has modest black...",NaN,California,Paso Robles,Central Coast,Cabernet Sauvignon,Castle Rock,31.0,NaN,Paso Robles Central Coast
2,US,"This lovely wine captures the floral, perfumed...",NaN,Oregon,Willamette Valley,,Gewürztraminer,Château Bianca,35.0,NaN,Willamette Valley
3,Portugal,"The aromas are the thing here, as so often wit...",NaN,Alentejano,NaN,NaN,Touriga Nacional,Herdade do Esporão,41.0,NaN,
4,Italy,"This is an interesting, outright strange wine ...",Natì,Southern Italy,Pompeiano,NaN,Coda di Volpe,Sorrentino,37.0,natì,Pompeiano
...,...,...,...,...,...,...,...,...,...,...,...
120739,Italy,This is a lovely Sauvignon with very character...,NaN,Northeastern Italy,Friuli,NaN,Sauvignon,Plozner,40.0,NaN,Friuli
120740,Spain,"Opens somewhat clipped and choppy, with citrus...",Reserva,Northern Spain,Rioja,NaN,Tempranillo Blend,Ramón Bilbao,41.0,reserve,Rioja
120741,Australia,Surprisingly complete and harmonious for a Rie...,NaN,South Australia,Adelaide Hills,NaN,Riesling,Wagtail,26.0,NaN,Adelaide Hills
120742,Germany,Broad aromas of ripe yellow fruit and a hint o...,Spätlese,Mosel,NaN,NaN,Riesling,Pamona Cellars,26.0,spätlese,


In [41]:
singular_region_complete = set(df["region_complete"])
len(singular_region_complete)

1211

In [43]:
import time
singular_region_complete_with_latlong = []
for region in singular_region_complete:
    time.sleep(1.0001)
    print(region)
    loc = locator.geocode(region)
    print(loc)
    if loc != None:
        singular_region_complete_with_latlong.append((region, loc.latitude, loc.longitude))

Jerez  
Jerez de la Frontera, Campiña de Jerez, Cádiz, Andalucía, España
Vinos de Madrid  
El Anciano Rey de los Vinos, 19, Calle de Bailén, Palacio, Centro, Madrid, Área metropolitana de Madrid y Corredor del Henares, Comunidad de Madrid, 28013, España
Mazis-Chambertin  
Climat de Bourgogne - Grand Cru - Mazis-Chambertin, Plantigone, Gevrey-Chambertin, Beaune, Côte-d'Or, Bourgogne-Franche-Comté, France métropolitaine, 21220, France
Côtes de Frontonnais  
None
Barbera del Monferrato Superiore  
None
Crémant de Limoux  
None
Touraine Amboise  
Route de Civray de Touraine, Amboise, Loches, Indre-et-Loire, Centre-Val de Loire, France métropolitaine, 37400, France
Moscadello di Montalcino  
None
Valencia  
València, Comarca de València, València / Valencia, Comunitat Valenciana, España
Tulum Valley  
Zona Arqueológica de Tulum, Avenida Ruinas Tulum, Tulum, Quintana Roo, 77764, México
Chianti Colli Fiorentini  
None
Martina  
La Martina, Municipio de General Roca, Departamento General Roca,

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Vin+de+Pays+de+la+M%C3%A9diterran%C3%A9e++&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)",))

from geopy.geocoders import Nominatim
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("Pompeiano")
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

df["Latitude"] = df["region_complete"].apply(lambda r: locator.geocode(r).latitude if pd.isna(locator.geocode(r)) == False else "")
df["Longitude"] = df["region_complete"].apply(lambda r: locator.geocode(r).longitude if pd.isna(locator.geocode(r)) == False else "")

In [ ]:
df

In [19]:
df.to_csv("competition_dataset/mine_dev.csv",index=False)